In [1]:
!python --version

Python 3.7.11


In [2]:
!pip install -q -U pip
!pip install -q numpy
!pip install -q pandas
!pip install -q ckiptagger
!pip install -q tqdm
!pip install -q tensorflow==1.14.0
!pip install -q ipywidgets

     |████████████████████████████████| 1.6 MB 8.4 MB/s 
     |████████████████████████████████| 109.3 MB 46 kB/s 
     |████████████████████████████████| 3.1 MB 43.5 MB/s 
     |████████████████████████████████| 50 kB 4.8 MB/s 
     |████████████████████████████████| 488 kB 74.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.14.0 which is incompatible.


In [3]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [5]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [6]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [7]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [8]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [9]:
from ckiptagger import data_utils
data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
1.88GB [00:19, 96.6MB/s]


In [10]:
ws = WS('./data/')
pos = POS('./data/')

In [11]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title]) # pass list to ws to parse the entire title
    word_p = pos(word_s)  # tag part of speech from the word_s variable
    # END YOUR CODE
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

  0%|          | 0/1800 [00:00<?, ?it/s]

In [12]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title])
    word_p = pos(word_s)
    # END YOUR CODE
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

  0%|          | 0/600 [00:00<?, ?it/s]

In [13]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

In [14]:
train_title_cuts[120][1][1]

'Nb'

# Bag of Words (BOW)

In [15]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
# YOUR CODE HERE
n = 0
for title in train_title_cuts:
  for word, pos in train_title_cuts[title]:
    if word in word2index:
      continue #avoid repeating words
    word2index[word] = n
    index2word[n] = word
    n += 1
# END YOUR CODE

In [16]:
word2index['溫暖']

1520

In [17]:
index2word[1520]

'溫暖'

In [18]:
def get_bow_vector(pairs, word2index):
    # YOUR CODE HERE
    vector = np.zeros(len(word2index))
    for word, flag in pairs: # +1 at the dictionary entry where the word is present 
    # (and that calculating cosine similarity has to be arrays of the same length)
      if word in word2index:
        vector[word2index[word]] += 1
    # END YOUR CODE
    return vector

In [19]:
test = get_bow_vector(train_title_cuts[120], word2index)

In [20]:
test, len(test)

(array([0., 0., 0., ..., 0., 0., 0.]), 6690)

# 排除較無意義的詞性

In [21]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['魅族', '萬孚', '尤文', '朱嘯虎', '阿木木歷', '萊昂納德', '大豐', '阿里', '騎士隊', '韋德', '烏迪內斯', '建成', '張', '老馬', '丁寧', '海王星', '泰倫盧', '魯能', 'J羅', '李盈瑩', '宋鴻兵', '費基爾', '楊柳夏', '余承東', '格力', '德羅巴', '王昭君', '呂布', '美宇', '納米 華', '海富通', '西游', '尤納斯', '羅傑斯', '德甲', '史森明', '弗格森', '盼保', '亞亞·圖雷', '許', '靜懿', '溫格', '男乒', '雪迪龍', '中金', '德羅', '東決', '碧生源', '龍星戰', '雷軍', '巴爾韋德', '安德森', '英特爾', '溫氏', '伊藤美誠', '馬克思', '德羅贊', '加威廉', '拳皇', '樂福', '騰訊', '安永', '李謹詺', '周雲傑', '章澤天', '王者榮', '施之皓', '皮爾洛', '馬雲', '瑞和寶', '人民日報', '克洛普', '黃紫昌', '詹黑皮爾斯', '斯諾克', '微醫', '蘇群', '建聯', '波爾', '2018世界杯', '侯森', '詹姆斯', '白起', '劉詩雯', '萊萬', '堪比比薩', '里坦克', '何雯娜', '羅現', '索尼', '美年達', '璞麗', '賈躍亭', '藍星', '姚勁波', '泰森', '華為', '張本智和', '泰山', '遼足']
VC : ['駕駛', '到', '分擔', '絕殺', '避開', '打野', '迎接', '救贖', '撿到', '面試', '保', '淘汰', '實現', '接受', '觀賞', '認養', '解綁', '攪亂', '嫁', '踢', '擴種', '拋棄', '引爆', '穩住', '翻譯', '造出', '發展', '分', '栽種', '攀比', '丟棄', '構建', '學', '上', '娶', '直播', '逗', '防治', '買', '放棄', '簽中', '纏', '產出', '佔領', '移栽', '獨造', '提升到', '玩', '對待', '引來', '開出', '贊取', '刷完', '懵逼', '護', '吊

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [22]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
        # YOUR CODE HERE
        # discard all punctuations and other words that do not aid in understanding
        "COLONCATEGORY", "COMMACATEGORY", "DASHCATEGORY", "DOTCATEGORY", "ETCCATEGORY", "EXCLAMATIONCATEGORY", "PARENTHESISCATEGORY",
        "PAUSECATEGORY", "QUESTIONCATEGORY", "SEMICOLONCATEGORY", "SPCHANGECATEGORY", "WHITESPACE",
        "Caa", "Cab", "Cba", "Cbb",
        "Di", "Dk", "DM", 
        "I", "DE", "SHI", "P", "T",
        "Nep", "Ng", "Nh", "V_2"
        # END YOUR CODE
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [23]:
import sklearn.metrics.pairwise
def cosine_similarity(bow1, bow2):
    # YOUR CODE HERE
    len_bow1 = (bow1 **2).sum() **(1/2)
    len_bow2 = (bow2 **2).sum() **(1/2)
    similarity = np.sum(bow1*bow2) / (len_bow1*len_bow2)
    # END YOUR CODE
    return similarity

In [24]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [25]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [26]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [27]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00333333, 0.00333333,
        0.00333333]),
 '體育': array([0.04      , 0.01333333, 0.00333333, ..., 0.        , 0.        ,
        0.        ])}

# Group mean vector: 測試

In [28]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [29]:
# prediction
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)
    total_news = sum(counter[num] for num in counter)
    acc = counter[group]/ total_news
    print("accuracy: {:.2f}".format(acc))

predict 體育 :  Counter({'體育': 72, '遊戲': 8, '財經': 6, '農業': 5, '旅遊': 4, '科技': 3})
accuracy: 0.73
predict 財經 :  Counter({'財經': 67, '科技': 21, '農業': 8, '遊戲': 6, '旅遊': 5, '體育': 3})
accuracy: 0.61
predict 科技 :  Counter({'科技': 60, '財經': 15, '體育': 10, '農業': 6, '遊戲': 6, '旅遊': 2})
accuracy: 0.61
predict 旅遊 :  Counter({'旅遊': 70, '農業': 9, '財經': 5, '科技': 2, '遊戲': 2, '體育': 1})
accuracy: 0.79
predict 農業 :  Counter({'農業': 68, '旅遊': 9, '科技': 6, '體育': 5, '財經': 4, '遊戲': 2})
accuracy: 0.72
predict 遊戲 :  Counter({'遊戲': 76, '體育': 8, '旅遊': 8, '科技': 7, '財經': 3, '農業': 3})
accuracy: 0.72
